In [1]:
import pandas as pd
import DataLoading
import NN_classes
import NN_evaluation
import torch
import matplotlib.pyplot as plt
import numpy as np
from time import perf_counter

In [3]:
sc = "sc01"
period = "2030"
folder = "../Data/3-bus_AC_12w_ext_o_dummy_LCOE"
#folder = "../Data/9n_AC_12w_ext_o_dummy_LCOE"
#folder = "../Data/RTS24_AC_12w_ext_o_dummy_LCOE/"

all_executions = DataLoading.list_executions(folder=folder,per = period,sc=sc)
te_s = 0.3
val_s = 0.4
dfs_in, dfs_out, dfs_inter = DataLoading.load_data_ext_out(folder, all_executions, period, sc, ["PowerFlow"])
dfs_inter_j = DataLoading.join_frames_inter_layer(dfs_inter)
dfs_inter_j = DataLoading.trim_columns_to_common(dfs_inter_j)
# Convert to pytorch tensors
ts_in, ts_out, ts_inter = DataLoading.split_tr_val_te_ext_out(dfs_in, dfs_out, dfs_inter_j, all_executions, te_s, val_s)
d_ft_in, d_ft_out, d_ft_inter,maxs = DataLoading.concat_and_normalize_ext_out(ts_in, ts_out, ts_inter, all_executions)



input_f_sc01_Network_Existing_Generation_Full_2030.csv
8
input_f_sc01_Network_Line_In_Node_1_Node_2_cac1_2030.csv
8
input_f_sc01_Network_Line_In_Node_1_Node_2_cac2_2030.csv
8
input_f_sc01_Network_Line_In_Node_1_Node_2_cac3_2030.csv
8
input_f_sc01_Network_Line_In_Node_1_Node_3_cac1_2030.csv
8
input_f_sc01_Network_Line_In_Node_1_Node_3_cac2_2030.csv
8
input_f_sc01_Network_Line_In_Node_1_Node_3_cac3_2030.csv
8
input_f_sc01_Network_Line_In_Node_2_Node_3_cac1_2030.csv
8
input_f_sc01_Network_Line_In_Node_2_Node_3_cac2_2030.csv
8
input_f_sc01_Network_Line_In_Node_2_Node_3_cac3_2030.csv
8


In [5]:
sc = "sc01"
period = "2030"
folder = "../Data/3-bus_AC_fy_ext_o_dummy_LCOE/"
#folder = "../Data/9n_AC_fy_ext_o_dummy_LCOE/"
#folder = "../Data/RTS24_AC_fy_ext_o_dummy_LCOE/"

all_executions = DataLoading.list_executions(folder=folder,per = period,sc=sc)
dfs_in_fy = DataLoading.load_data_input_only(folder,all_executions,period,sc)

input_size = dfs_in["Network_Existing_Generation_Full"].shape[1]
inter_size = dfs_inter_j["Network_Existing_Generation_Full"].shape[1]

input_f_sc01_Network_Existing_Generation_Full_2030.csv
8
input_f_sc01_Network_Line_In_Node_1_Node_2_cac1_2030.csv
8
input_f_sc01_Network_Line_In_Node_1_Node_2_cac2_2030.csv
8
input_f_sc01_Network_Line_In_Node_1_Node_2_cac3_2030.csv
8
input_f_sc01_Network_Line_In_Node_1_Node_3_cac1_2030.csv
8
input_f_sc01_Network_Line_In_Node_1_Node_3_cac2_2030.csv
8
input_f_sc01_Network_Line_In_Node_1_Node_3_cac3_2030.csv
8
input_f_sc01_Network_Line_In_Node_2_Node_3_cac1_2030.csv
8
input_f_sc01_Network_Line_In_Node_2_Node_3_cac2_2030.csv
8
input_f_sc01_Network_Line_In_Node_2_Node_3_cac3_2030.csv
8


In [23]:
hyperloop_name = "3-bus_AC_12w_dummy_0.3_v0.4_PF_LCOE_0_10"
#hyperloop_name = "9n_AC_12w_dummy_0.3_v0.4_PF_LCOE_0_37"
# hyperloop_name = "RTS24_AC_12w_dummy_0.3_v0.4_PF_sa_rand_0_40"

folder_exports = "3-bus_AC_fy_ext_o_dummy_LCOE"


df_losses = pd.read_csv(f"ResultsClusterRuns/Loss_results_csv/{hyperloop_name}.csv",index_col=0)
#df_losses = pd.read_csv(f"Loss_results_csv/{hyperloop_name}.csv",index_col=0)


In [26]:
df_extraction_time = pd.DataFrame()
i=0
for loss in ["Te_l_mae","Te_l_t_mse"]:
    for xth_best in [1,2]:
        ##Find and: load model
        for alpha in [0.0,0.16]:
            f = df_losses.alpha ==alpha
            # f = df_losses.Model_type == "(3, 2)"
            loss_to_sort = loss
            row = NN_evaluation.find_xthbest_model_params_from_df(df_losses[f],loss_to_sort,xth_best)
            m = NN_evaluation.create_model_and_load_state_from_row(row,input_size,inter_size,hyperloop_name,cluster_run=True)

            ##Uste model for estimates
            df_all_exec = pd.DataFrame()
            for ex in all_executions:
                t_start = perf_counter()
                prediction = NN_evaluation.get_NN_estimates_from_dfs_in(m,ex,dfs_in_fy,maxs)
                t_elapsed = perf_counter() - t_start
                #actual= get_actual(ex,dfs_out) 
                df_all_exec[ex] = prediction

            mpd = model_params_dict = NN_evaluation.extract_model_params_from_row(row)
            keys = ["Model_type","nb_e","lr","dor","np","ro","bs","alpha","MAE", "Min_val"]
            name = f"Estimated_OC_fy_{mpd[keys[0]]}h_{mpd[keys[1]]}e_{mpd[keys[2]]}lr_{mpd[keys[3]]}dor_{mpd[keys[4]]}np_{mpd[keys[5]]}ro_{mpd[keys[6]]}bs_{mpd[keys[7]]}ill_{mpd[keys[8]]}MAE_{mpd[keys[9]]}Min_val"
            #name = f"Estimated_OC_fy_{mpd[keys[0]]}h_{mpd[keys[1]]}e_{mpd[keys[2]]}lr_{mpd[keys[3]]}dor_{mpd[keys[4]]}np_{mpd[keys[5]]}ro_{mpd[keys[6]]}bs_{mpd[keys[7]]}ill_{mpd[keys[8]]}MAE"
            time_row = pd.DataFrame({"Model":name,"Evaluation Time": t_elapsed},index = [i] )
            i+=1
            df_extraction_time = pd.concat([df_extraction_time,time_row])
            #Write exports to csv
            #df_all_exec.to_csv(f"NN_estimate_exports/{folder_exports}/{name}.csv")
df_extraction_time.to_csv(f"NN_estimate_exports/{folder_exports}/evaluation_times.csv")

[16, 24, 5, 3]
[16, 24, 5, 3]
[16, 24, 5, 3]
[16, 24, 5, 3]
[16, 24, 5, 3]
[16, 24, 5, 3]
[16, 24, 5, 3]
[16, 24, 5, 3]


In [15]:
df_extraction_time

,Model,Evaluation Time
0,"Estimated_OC_fy_(3, 0)h_128e_0.0025lr_0.0dor_0...",0.003910
1,"Estimated_OC_fy_(3, 0)h_128e_0.000625lr_0.05do...",0.003271
2,"Estimated_OC_fy_(3, 0)h_128e_0.0025lr_0.0dor_0...",0.004327
3,"Estimated_OC_fy_(3, 0)h_128e_0.000625lr_0.05do...",0.003948


In [7]:
ex = all_executions[0]

df_all_exec = pd.DataFrame()
for ex in all_executions:
    prediction = get_NN_estimates_from_dfs_in(m,ex,dfs_in)
    #actual= get_actual(ex,dfs_out) 
    df_all_exec[ex] = prediction

mpd = model_params_dict = extract_model_params_from_row(row)
keys = ["Model_type","nb_e","lr","dor","np","ro","bs","alpha","MAE"]
name = f"Estimated_OC_12w_{mpd[keys[0]]}h_{mpd[keys[1]]}e_{mpd[keys[2]]}lr_{mpd[keys[3]]}dor_{mpd[keys[4]]}np_{mpd[keys[5]]}ro_{mpd[keys[6]]}bs_{mpd[keys[7]]}ill_{mpd[keys[8]]}MAE"
df_all_exec.to_csv(f"NN_estimate_exports/{name}.csv")

IndexError: index 0 is out of bounds for axis 0 with size 0